# Boosting Models

**Date:** 11/11/2025

* https://xgboost.readthedocs.io/en/stable/tutorials/model.html
* https://lightgbm.readthedocs.io/en/stable/

In [3]:
#pip install xgboost lightgbm catboost

# 1. Libraries

In [4]:
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

# 1.0 Boosting, XGBoost y LightGBM para Modelos de Regresión

## 1.0.1 Boosting para Regresión

### Idea general

**Boosting** es una técnica de *ensemblado secuencial* que combina **muchos modelos débiles** (por lo general, pequeños árboles de decisión llamados *stumps*) para formar un **modelo fuerte y preciso**.

A diferencia de los **Random Forests**, donde los árboles se entrenan **en paralelo e independientemente**, en el **Boosting** los árboles se entrenan **uno después de otro**, y cada nuevo árbol **corrige los errores del anterior**.

---

### Intuición paso a paso

1. Se comienza con una predicción inicial simple, por ejemplo:
   $$
   \hat{y}^{(0)} = \bar{y} = \frac{1}{N}\sum_{i=1}^{N} y_i
   $$
   (el promedio de los valores reales).

2. En cada iteración $ m = 1, 2, \dots, M $:
   - Se calculan los **residuos** (errores actuales):
     $$
     r_i^{(m)} = y_i - \hat{y}_i^{(m-1)}
     $$
   - Se entrena un **nuevo árbol de regresión** para predecir estos residuos.
   - Se actualiza la predicción del modelo:
     $$
     \hat{y}_i^{(m)} = \hat{y}_i^{(m-1)} + \eta \, h_m(x_i)
     $$
     donde:
     - $ h_m(x_i) $ es la predicción del árbol $ m $,
     - $ \eta $ (learning rate) controla cuánto peso se da a cada nuevo árbol.

3. El modelo final es la suma ponderada de los árboles:
   $$
   \hat{f}(x) = \sum_{m=1}^{M} \eta \, h_m(x)
   $$

---

### Características clave del Boosting

- Cada árbol intenta **corregir los errores residuales** del modelo anterior.  
- Se da **más peso a las observaciones mal predichas**.  
- El parámetro **learning rate ($ \eta $)** controla la velocidad de aprendizaje:
  - Valores pequeños (p. ej. 0.05) → aprendizaje más lento pero más estable.  
  - Valores grandes → riesgo de sobreajuste.

---

### Ventajas y desventajas

| Ventajas | Desventajas |
|-----------|--------------|
| Alta precisión | Más lento de entrenar |
| Menor sesgo que Random Forest | Sensible al ruido |
| Captura relaciones complejas | Muchos hiperparámetros |

---

## 2 XGBoost (Extreme Gradient Boosting)

### Qué es

**XGBoost** es una implementación optimizada del algoritmo de Boosting basada en el concepto de **Gradient Boosting**.  
Fue diseñada para ser **rápida, escalable y regularizada**, lo que la convierte en una de las herramientas más populares en *machine learning*.

---

### Gradient Boosting: concepto matemático

En lugar de ajustar los residuos directamente, XGBoost ajusta **la dirección del gradiente del error** respecto a las predicciones actuales.

Dado un conjunto de datos $(x_i, y_i)$ y una función de pérdida $ L(y_i, \hat{y}_i) $, el objetivo es minimizar:

$$
\text{Obj} = \sum_{i=1}^{N} L(y_i, \hat{y}_i) + \sum_{m=1}^{M} \Omega(f_m)
$$

donde:
- $ L $ mide el error de predicción (por ejemplo, MSE),
- $ \Omega(f_m) $ penaliza la complejidad de los árboles, para evitar sobreajuste.

---

### Actualización por gradiente

En cada iteración $ m $, XGBoost construye un árbol $ f_m(x) $ que aproxima el **gradiente negativo** de la función de pérdida:

$$
g_i^{(m)} = \frac{\partial L(y_i, \hat{y}_i^{(m-1)})}{\partial \hat{y}_i^{(m-1)}}
$$

Luego ajusta el árbol para minimizar una **aproximación de segundo orden** de la función de pérdida:

$$
\text{Obj}^{(m)} \approx \sum_{i=1}^{N} \left[ g_i^{(m)} f_m(x_i) + \frac{1}{2} h_i^{(m)} f_m^2(x_i) \right] + \Omega(f_m)
$$

donde $ h_i^{(m)} $ es la segunda derivada (Hessiano).

---

### Regularización

XGBoost incluye una función de penalización para controlar la complejidad de los árboles:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
$$

donde:
- $ T $ es el número de hojas,
- $ w_j $ son los pesos de cada hoja,
- $ \gamma $ y $ \lambda $ son parámetros de regularización.

Esto reduce el sobreajuste y mejora la generalización.

---

### Ventajas de XGBoost

* Muy rápido (usa paralelización y memoria eficiente).  
* Regularización integrada ($ L_1 $ y $ L_2 $). 
* Maneja datos faltantes de forma nativa.
*  Permite árboles profundos y controlados.  

---

## 3 LightGBM (Light Gradient Boosting Machine)

### Qué es

**LightGBM** es otra implementación de *gradient boosting*, desarrollada por Microsoft.  
Se diseñó para ser **aún más rápida y eficiente** que XGBoost, especialmente en conjuntos de datos grandes y de alta dimensión.

---

### Diferencia clave: método de crecimiento del árbol

Mientras que XGBoost crece los árboles **nivel por nivel (level-wise)**,  
**LightGBM crece los árboles hoja por hoja (leaf-wise)**, eligiendo siempre la hoja con la mayor ganancia de información.

* **XGBoost (level-wise):**
- Expande todos los nodos del mismo nivel.
- Más equilibrado, menor sobreajuste.

* **LightGBM (leaf-wise):**
- Expande la hoja con mayor ganancia.
- Puede generar árboles más profundos y precisos.
- Mayor riesgo de sobreajuste (controlado con `max_depth`).

---

### Optimizaciones adicionales

1. **Histogram-based learning:**  
   Convierte los valores continuos en *bins* (intervalos), reduciendo el costo computacional.

2. **Gradient-based One-Side Sampling (GOSS):**  
   Usa solo una fracción de las muestras con grandes gradientes (más informativas).

3. **Exclusive Feature Bundling (EFB):**  
   Combina características mutuamente excluyentes en una sola, reduciendo la dimensionalidad.

---

### Fórmula de actualización (similar a XGBoost)

$$
\hat{y}_i^{(m)} = \hat{y}_i^{(m-1)} + \eta \, f_m(x_i)
$$

pero con un método de partición más eficiente y un cálculo de histograma optimizado.

---

### Ventajas de LightGBM

* Entrenamiento extremadamente rápido.  
* Escala muy bien con millones de filas y columnas.  
* Alta precisión con datos grandes.  
* Bajo consumo de memoria.

---

### Desventajas

* Puede sobreajustar si los datos son pequeños.  
* Menos interpretable.  
* Parámetros más sensibles que en XGBoost.

---

## Resumen

| Característica | Boosting Clásico | XGBoost | LightGBM |
|----------------|------------------|----------|-----------|
| Tipo de boosting | Residual | Gradient (2° orden) | Gradient (2° orden optimizado) |
| Regularización | No explícita | Sí (L1 y L2) | Sí |
| Crecimiento del árbol | Secuencial simple | Nivel por nivel | Hoja por hoja |
| Velocidad | Media | Rápido | Muy rápido |
| Escalabilidad | Media | Alta | Muy alta |
| Riesgo de overfitting | Medio | Bajo | Medio-Alto |
| Ideal para | Datasets medianos | Datos estructurados | Big data/tablas muy grandes |

---

## Conclusión

- **Boosting:** combina modelos débiles secuencialmente para mejorar el rendimiento.  
- **XGBoost:** implementación optimizada del gradient boosting con regularización y segunda derivada.  
- **LightGBM:** versión aún más rápida y escalable, ideal para grandes volúmenes de datos.

Ambos (XGBoost y LightGBM) se han convertido en **estándares industriales** por su eficiencia, precisión y capacidad de generalización en problemas de regresión y clasificación.



## 1.1. XGBoost Model

In [5]:
# Cargamos datos de viviendas en California
datos = fetch_california_housing()
X, y = datos.data, datos.target

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
# Convertimos a formato DMatrix (optimizado para XGBoost)
xgb_dtrain = xgb.DMatrix(X_train, label=y_train)
xgb_dtest = xgb.DMatrix(X_test, label=y_test)

# Configuración del modelo
xgb_parameters = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.1  # Tasa de aprendizaje
}

# Entrenamiento
modelo = xgb.train(xgb_parameters, xgb_dtrain, num_boost_round=100)

# Predicción
y_train_preds = modelo.predict(xgb_dtrain)
print("Error Train (RMSE):", root_mean_squared_error(y_train, y_train_preds))

y_test_preds = modelo.predict(xgb_dtest)
print("Error Test (RMSE):", root_mean_squared_error(y_test, y_test_preds))


Error Train (RMSE): 0.3689490478304685
Error Test (RMSE): 0.47672034565362764


In [8]:
import lightgbm as lgb

In [9]:
lgb_dtrain = lgb.Dataset(X_train, y_train)
lgb_dtest = lgb.Dataset(X_test, y_test)

In [10]:
lgb_parameters = {
    "objective": "regression",
    "num_leaves": 31,
    "learning_rate": 0.05
}

In [11]:
lgb_model = lgb.train(lgb_parameters,lgb_dtrain, num_boost_round=100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000918 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.071947


In [13]:
# Predicción
y_train_preds = lgb_model.predict(X_train)
print("Error Train (RMSE):", root_mean_squared_error(y_train, y_train_preds))

y_test_preds = lgb_model.predict(X_test)
print("Error Test (RMSE):", root_mean_squared_error(y_test, y_test_preds))


Error Train (RMSE): 0.4481941166091315
Error Test (RMSE): 0.4882208769040563


## 3. CatBoost

https://catboost.ai/docs/en/concepts/python-reference_catboostregressor

In [69]:
from catboost import CatBoostRegressor

# Initialize the CatBoostRegressor with RMSE as the loss function
cb_model = CatBoostRegressor(
    loss_function='RMSE',
    max_depth=8,
    learning_rate=0.1,
    #n_estimators=300,
    num_boost_round=300,
    min_data_in_leaf=10,
    )

# Fit the model on the training data with verbose logging every 100 iterations
cb_model.fit(X_train, y_train, verbose=100)

0:	learn: 1.0897903	total: 19ms	remaining: 5.69s
100:	learn: 0.4530808	total: 529ms	remaining: 1.04s
200:	learn: 0.3907115	total: 1.16s	remaining: 573ms
299:	learn: 0.3532218	total: 1.65s	remaining: 0us


In [70]:
# Predicción
y_train_preds = cb_model.predict(X_train)
print("Error Train (RMSE):", root_mean_squared_error(y_train, y_train_preds))

y_test_preds = cb_model.predict(X_test)
print("Error Test (RMSE):", root_mean_squared_error(y_test, y_test_preds))

Error Train (RMSE): 0.35322184789597055
Error Test (RMSE): 0.4525225693589356
